In [ ]:
ls /home/silvio/dataset/passengers/airline-passengers.csv

# Análise temporal

In [ ]:
import pandas as pd
import numpy as np

from dateutil import parser
from datetime import datetime

import matplotlib.pyplot as plt

from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error

from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm

from statsmodels.tsa.filters.hp_filter import hpfilter

from statsmodels.tsa.filters.hp_filter import hpfilter
from scipy import signal
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.seasonal import seasonal_decompose 

plt.rcParams.update({'figure.figsize':(9,3), 'figure.dpi':120})
import warnings
warnings.filterwarnings('ignore')

# Series estacionárias
* A média da série não deve ser uma função do tempo, e sim uma constante. 
* Quando uma série temporal é estacionária, pode ser mais fácil modelar. 

* A distribuição de nascimento de aves se mostra constante ao passar do tempo
* A distribuição de passageiros mostra duas características claras de não-estacionaridade
    * tendência: número crescente de passageiros
    * repetições de padrões em períodos específicos: meses de férias apresentam um pico de passageiros

## Análise gráfica
* Série mostrando observação de nascimento de aves
* Série mostrando quantidade de passageiros por mês em uma cia área

In [ ]:
birds = pd.read_csv('/home/silvio/dataset/daily.csv', header=0, index_col=0)
birds.plot(figsize=(20,10))

In [ ]:
airPassagers = pd.read_csv('/home/silvio/dataset/passengers/airline-passengers.csv', header=0, index_col=0)
airPassagers.plot(figsize=(20,10))

In [ ]:
airPassagers

## Estatistica Resumida

* Segmentando a série temporal in partições menores e comparando a média e variância de cada grupo
* Se a diferença é significativa a série é não-estacionária

* A seguir comparamos o histograma das duas séries

In [ ]:
birds.hist()

## Obtendo média e variância de duas partes distinstas da série
* Média e variância da base de aves é similar em diferentes partes
* Média e variância da base de Passengers é bem diferente em diferentes partes

In [ ]:
X = birds.values
split = round(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('media 1=%f, media 2=%f' % (mean1, mean2))
print('variancia 1=%f, variancia 2=%f' % (var1, var2))

In [ ]:
airPassagers.hist()

## Obtendo média e variância de duas partes distintas da série

In [ ]:
X = airPassagers.values
split = round(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('media 1=%f, media 2=%f' % (mean1, mean2))
print('variancia 1=%f, variancia 2=%f' % (var1, var2))

## Testando em escala log (passengers)

## Comparando resultado da avaliação estatística

* Dados de nascimento de aves
    * A média e variância das observações mostra um comportamente similar , rementendo a distribuição gaussiana 
    * Visualmente o histograma também reflete tal similaridade
* Dados de passageiros de avião
    * Para esse dataset a conclusão é que a distribuição é dependente no tempo, mostrando que é não-estacionária

# Teste Estatístico

* testes estatísticos são mais confiáveis para classisficar uma série com estacionária ou não-estacionária

* Teste Dickey-Fuller


## teste adfuller para base de aves

In [ ]:
X = birds.values
result = adfuller(X)
print('ADF: %f' % result[0])
print('p-value: %f' % result[1])
print('valores criticos:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

## teste adfuller para base cia aerea

In [ ]:

X = airPassagers.values
result = adfuller(X)
print('ADF: %f' % result[0])
print('p-value: %f' % result[1])
print('valores criticos:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

## teste adfuller para base cia aerea (Escala Log)

In [ ]:
X = airPassagers.values
X = np.log(X)
result = adfuller(X)
print('ADF: %f' % result[0])
print('p-value: %f' % result[1])
print('valores criticos:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

## Hodrick-Prescott (HP) filter

# Tendencia (trend)
* Uma tendência é um padrão observado ao longo de um período de tempo e representa a taxa média de mudança em relação ao tempo. 

* Uma tendência geralmente mostra a tendência dos dados de aumentar / tendência de alta ou diminuir / tendência de baixa durante o longo prazo.

hp_filter separa tendências em um dataset

# no dataset airPassagers observamos que o número de passageiros cresce a cada ano.

In [ ]:
_,trend = hpfilter(airPassagers['Passengers'])#, lamb=1600) 
trend.plot(figsize=(15,6)).autoscale(axis='x',tight=True)

# no dataset Birds existe um pico de crescimento e um perído longo de estabilidade

In [ ]:
_,trend = hpfilter(birds['Births'])#, lamb=1600) 
trend.plot(figsize=(15,6)).autoscale(axis='x',tight=True)

# detrend

Detrending é o processo de 
remover uma tendência dos dados da série temporal 
ou mencionar uma mudança na média ao longo do tempo. 
Está aumentando continuamente ou diminuindo ao longo do tempo. 

A identificação, modelagem e até mesmo a remoção de dados de tendência de conjuntos de dados de séries temporais podem ser benéficas.

* usando função diff do pandas

In [ ]:
diff = airPassagers.Passengers.diff()
plt.figure(figsize=(15,6))
plt.plot(diff)
plt.title('Detrending usando diferenciação', fontsize=16) 
plt.xlabel('Mes')
plt.ylabel('Passageiros')
plt.show()

# detrend usando fução detrend do scipy

* Um sinal é outra forma de dados de série temporal. 

* Cada sinal aumenta ou diminui em uma ordem diferente. Usando a biblioteca SciPy, isso pode remover a tendência linear dos dados do sinal. 

* Signal.detrend é um submódulo do SciPy usado para remover uma tendência linear ao longo de um eixo dos dados.

In [ ]:


detrended = signal.detrend(airPassagers.Passengers.values)
plt.figure(figsize=(15,6)) 
plt.plot(detrended) 
plt.xlabel('Valor')
plt.ylabel('Frequencia')
plt.title('Detrending usando Scipy Signal', fontsize=16) 
plt.show() 

In [ ]:

detrended = signal.detrend(birds.Births.values)
plt.figure(figsize=(15,6)) 
plt.plot(detrended) 
plt.xlabel('mes')
plt.ylabel('Nascimentos')
plt.title('Detrending usando Scipy Signal', fontsize=16) 
plt.show()

Um filtro HP também é usado para diminuir uma série de tempo e suavizar os dados. É usado para remover flutuações de curto prazo. O código a seguir mostra um exemplo de remoção de distorção do filtro HP.

In [ ]:
_,trend = hpfilter(airPassagers['Passengers'])

detrended = airPassagers.Passengers - trend
plt.figure(figsize=(15,6))
plt.plot(detrended)
plt.title('Detrending usando HP Filter', fontsize=16) 
plt.xlabel('mes')
plt.ylabel('passageiros') 
plt.show()

In [ ]:

_,trend= hpfilter(birds.Births)

plt.figure(figsize=(15,6))
plt.plot(trend)
plt.title('Detrending usando HP Filter', fontsize=16) 
plt.xlabel('mes')
plt.ylabel('Nascimentos') 
plt.show()

* identificando autocorrelação usando Pandas

In [ ]:

autocorrelation_plot(airPassagers.Passengers.tolist())

In [ ]:
autocorrelation_plot(birds.Births.tolist())

In [ ]:
plot_acf(birds.Births)

In [ ]:
plot_acf(birds.Births.diff().dropna())

In [ ]:
plot_acf(airPassagers.Passengers)

In [ ]:
plot_acf(airPassagers.Passengers.diff().dropna())

In [ ]:
plot_acf(airPassagers.Passengers.diff().diff().dropna())

# Decompondo uma série temporal em seus componentes

* Decomposição é um método usado para isolar os dados da série temporal em diferentes elementos, como tendências, sazonalidade, variância cíclica e resíduos. Podemos aproveitar a decomposição sazonal de um modelo de estatísticas

* decompor os dados em suas partes constituintes, considerando as séries como aditivas ou multiplicativas.
    * Tendências aumento ou diminuição no valor dos dados ts.
    * Sazonalidade repetir um ciclo de curto prazo de dados ts.
    * Variações cíclicas  flutuação nas tendências longas dos dados ts.
    * Residuais variação irregular dos dados ts.

# transformando índice em data

In [ ]:
tdi = pd.DatetimeIndex(airPassagers.index)
airPassagers.set_index(tdi, inplace=True)
airPassagers.index.name = 'datetimeindex'

In [ ]:


result = seasonal_decompose(airPassagers['Passengers'], model='add') 
result.plot()

In [ ]:
result = seasonal_decompose(airPassagers['Passengers'], model='mul') 
result.plot()

In [ ]:
tdi = pd.DatetimeIndex(birds.index)
birds.set_index(tdi, inplace=True)
birds.index.name = 'datetimeindex'

result = seasonal_decompose(birds['Births'], model='add') 
result.plot()

In [ ]:
result = seasonal_decompose(birds['Births'], model='mul') 
result.plot()

# Dividindo a base em treino e teste
* Uma estratégia é definir a parte inicial como treino e o fim da sério como teste

In [ ]:
train = airPassagers[:int(0.7*(len(airPassagers)))] 
valid = airPassagers[int(0.7*(len(airPassagers))):]
print(train.shape)
print(valid.shape)
train['Passengers'].plot() 
valid['Passengers'].plot()

In [ ]:
#10,2,2 - passengers

model = ARIMA(train, order=(15, 1, 1))  
fitted = model.fit()

# Forecast
fc, _, _ = fitted.forecast(valid.shape[0], alpha=0.05)  # 95% conf

fc_series = pd.Series(fc, index=valid.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(train, label='Treino')
plt.plot(valid, label='Teste')
plt.plot(fc_series, label='Predição')
plt.title('Comparando Predição e Original')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
residuals = DataFrame(fitted.resid)
residuals.plot()
pyplot.show()
# density plot of residuals
residuals.plot(kind='kde')
pyplot.show()

# MSE

In [ ]:
fc, _, _ = fitted.forecast(valid.shape[0], alpha=0.05)  # 95% conf
mse=mean_squared_error(valid.Passengers, fc, squared=True)
mse

In [ ]:
fc, _, _ = fitted.forecast(valid.shape[0], alpha=0.05)  # 95% conf
mse=mean_squared_error(valid.Births, fc, squared=True)
mse

# Separando Treino e Teste passaros

In [ ]:
train = birds[:int(0.85*(len(birds)))] 
valid = birds[int(0.85*(len(birds))):]
print(train.shape)
print(valid.shape)
train['Births'].plot() 
valid['Births'].plot()